# Lido sandbox

## Init Lido contract

In [1]:
from lido import Lido

lido = Lido()

lido.balance = 1 * 10**18
lido.initialize()

lido.submit('user 1', 64 * 10**18)
lido.submit('user 3', 32 * 10**18)

None

## Init staking modules

In [2]:
_, router = lido.locator.staking_router

router.add_staking_module(name='curated', staking_module_address='curated module', target_share=10000, staking_module_fee=500, treasury_fee=500)
router.add_staking_module(name='community', staking_module_address='community module', target_share=10000, staking_module_fee=700, treasury_fee=300)

operators = 5
submitted_keys_per_operator = 10

# init curated module
curated_module = router.get_staking_module_instance('curated module')
for no_index in range(operators):
    curated_module.add_node_operator('curated operator %d' % (no_index), 'curated operator %d' % (no_index))
    curated_module.add_signing_keys(
        no_index, 
        [["curated key %d.%d" % (no_index, key_index), "curated signature %d.%d" % (no_index, key_index)] for key_index in range(submitted_keys_per_operator)]
    )
    curated_module.set_node_operator_staking_limit(no_index, submitted_keys_per_operator)

# init community module
community_module = router.get_staking_module_instance('community module')
for no_index in range(operators):
    community_module.add_node_operator('community operator %d' % (no_index), 'community operator %d' % (no_index))
    community_module.add_signing_keys(
        no_index, 
        [["community key %d.%d" % (no_index, key_index), "community signature %d.%d" % (no_index, key_index)] for key_index in range(submitted_keys_per_operator)]
    )
    community_module.set_node_operator_staking_limit(no_index, submitted_keys_per_operator)

curated module {0: [['curated key 0.0', 'curated signature 0.0'], ['curated key 0.1', 'curated signature 0.1'], ['curated key 0.2', 'curated signature 0.2'], ['curated key 0.3', 'curated signature 0.3'], ['curated key 0.4', 'curated signature 0.4'], ['curated key 0.5', 'curated signature 0.5'], ['curated key 0.6', 'curated signature 0.6'], ['curated key 0.7', 'curated signature 0.7'], ['curated key 0.8', 'curated signature 0.8'], ['curated key 0.9', 'curated signature 0.9']], 1: [['curated key 1.0', 'curated signature 1.0'], ['curated key 1.1', 'curated signature 1.1'], ['curated key 1.2', 'curated signature 1.2'], ['curated key 1.3', 'curated signature 1.3'], ['curated key 1.4', 'curated signature 1.4'], ['curated key 1.5', 'curated signature 1.5'], ['curated key 1.6', 'curated signature 1.6'], ['curated key 1.7', 'curated signature 1.7'], ['curated key 1.8', 'curated signature 1.8'], ['curated key 1.9', 'curated signature 1.9']], 2: [['curated key 2.0', 'curated signature 2.0'], ['cu

## Deposit

In [3]:
lido.deposit(2, 1) # 2 deposits to module 1
lido.deposit(1, 2) # 1 deposit to module 2

In [4]:
from helpers.print_table import print_table
from helpers.wei_to_eth import wei_to_eth

deposited_validators, cl_validators, cl_balance = lido.get_beacon_stat()

print_table(
    [[
        wei_to_eth(lido.total_supply()),
        wei_to_eth(lido.get_total_shares()),
        wei_to_eth(lido.get_buffered_ether()),
        wei_to_eth(cl_balance),
        cl_validators,
        deposited_validators,
    ]],
    [
        'total supply',
        'total shares',
        'buffered',
        'cl balance',
        'cl vals',
        'deposited vals'
    ]
)

total supply,total shares,buffered,cl balance,cl vals,deposited vals
97.0,97.0,1.0,0.0,0,3


## Stakers

In [5]:
from helpers.print_table import print_table

print_table(
    [[key, wei_to_eth(value), wei_to_eth(lido.get_pooled_eth_by_shares(value))] for key, value in lido._shares.items()],
    ['address', 'shares', 'steth']
)

address,shares,steth
0xdead,1.0,1.0
user 1,64.0,64.0
user 3,32.0,32.0


## Oracle report

In [6]:
from time import time
current_timestamp = int(time())

lido.handle_oracle_report(
    report_timestamp = current_timestamp,
    time_elapsed = current_timestamp - lido._last_report_timestamp,
    cl_validators = lido._deposited_validators,
    cl_balance = lido._deposited_validators * lido.DEPOSIT_SIZE,
    withdrawal_vault_balance = 2 * 10 ** 18,
    el_rewards_vault_balance = 2 * 10 ** 18,
    shares_requested_to_burn = 0,
    withdrawal_finalization_batches = [],
    simulated_share_rate = 0
)

None

AttributeError: 'Lido' object has no attribute 'shares'